In [20]:
import tensorflow as tf
import numpy as np

In [21]:
def reset_graph(seed=318):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [22]:
from tensorflow.examples.tutorials.mnist import input_data

In [23]:
mnist = input_data.read_data_sets('/Users/huwenyue/mnist2/',one_hot=True)

Extracting /Users/huwenyue/mnist2/train-images-idx3-ubyte.gz
Extracting /Users/huwenyue/mnist2/train-labels-idx1-ubyte.gz
Extracting /Users/huwenyue/mnist2/t10k-images-idx3-ubyte.gz
Extracting /Users/huwenyue/mnist2/t10k-labels-idx1-ubyte.gz


In [24]:
#image
img_height = 28
img_width = 28
img_channels = 1
img_classes = 10

#neural network
n_steps = img_height
n_inputs = img_width
n_neurons = 20
n_outputs = img_classes

learning_rate = 1e-4

In [25]:
reset_graph()

In [26]:
x = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.float32,[None,n_outputs])

In [27]:
#RNN
outputs,states = tf.nn.dynamic_rnn(tf.contrib.rnn.BasicRNNCell(n_neurons,activation=tf.nn.tanh),x,dtype=tf.float32)

#output layer
logits = tf.layers.dense(states,n_outputs,activation=tf.nn.softmax)


In [28]:
#loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits)
loss = tf.reduce_mean(cross_entropy)

In [29]:
#optimizer
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [30]:
#evaluation models
correct = tf.equal(tf.argmax(y,1),tf.argmax(logits,1))
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [31]:
n_epochs = 21
batch_size = 10

init = tf.global_variables_initializer()

#统计量
loss_summary = tf.summary.scalar("loss",loss)
acc_summary = tf.summary.scalar("acc",accuracy)
merged = tf.summary.merge_all()

#保存统计信息的磁盘文件
logdir = "/Users/huwenyue/mnist_logs/"
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())

#build check points
checkpoints_path = "/Users/huwenyue/mnist_checkpoints/"
saver = tf.train.Saver()

In [32]:
with tf.Session() as sess:
    sess.run(init)
    #saver.restore(sess,checkpoints_path)
    
    for epoch in range(n_epochs):
        #training
        x_batch, y_batch = mnist.train.next_batch(batch_size)
        x_batch_rnn = np.reshape(x_batch,[-1,n_steps,n_inputs])
        train_acc, _= sess.run([accuracy,train],feed_dict={x:x_batch_rnn,y:y_batch})
        
        #testing
        x_batch, y_batch = mnist.test.next_batch(batch_size)
        x_batch_rnn = np.reshape(x_batch,[-1,n_steps,n_inputs])
        [merged_str,test_acc] = sess.run([merged,accuracy],feed_dict={x:x_batch_rnn,y:y_batch})
        
        file_writer.add_summary(merged_str,epoch)
        print("epoch: ", str(epoch),"train_acc: ",str(train_acc),";test_acc: ",str(test_acc))
    saver.save(sess,checkpoints_path)
    
file_writer.close()

epoch:  0 train_acc:  0.1 ;test_acc:  0.2
epoch:  1 train_acc:  0.1 ;test_acc:  0.1
epoch:  2 train_acc:  0.2 ;test_acc:  0.3
epoch:  3 train_acc:  0.0 ;test_acc:  0.1
epoch:  4 train_acc:  0.1 ;test_acc:  0.1
epoch:  5 train_acc:  0.0 ;test_acc:  0.0
epoch:  6 train_acc:  0.0 ;test_acc:  0.0
epoch:  7 train_acc:  0.1 ;test_acc:  0.1
epoch:  8 train_acc:  0.0 ;test_acc:  0.0
epoch:  9 train_acc:  0.3 ;test_acc:  0.0
epoch:  10 train_acc:  0.3 ;test_acc:  0.1
epoch:  11 train_acc:  0.3 ;test_acc:  0.1
epoch:  12 train_acc:  0.2 ;test_acc:  0.2
epoch:  13 train_acc:  0.0 ;test_acc:  0.0
epoch:  14 train_acc:  0.0 ;test_acc:  0.0
epoch:  15 train_acc:  0.0 ;test_acc:  0.2
epoch:  16 train_acc:  0.2 ;test_acc:  0.2
epoch:  17 train_acc:  0.1 ;test_acc:  0.1
epoch:  18 train_acc:  0.1 ;test_acc:  0.1
epoch:  19 train_acc:  0.1 ;test_acc:  0.1
epoch:  20 train_acc:  0.0 ;test_acc:  0.3


In [33]:
!ls

AnacondaProjects   Downloads          Pictures           mnist2
Applications       Library            Public             mnist_checkpoints
Desktop            Movies             RNN.ipynb          mnist_logs
Documents          Music              anaconda3          约瑟夫环.ipynb
